In [5]:

print('This is a Program designed to handle IV-perovskite data enabling operating Origin Software readily.')
print('\nThis program will automatically import the highlight columns of the opened Excel file to draw IV-curve of PSCs with the best PCE.')
print('\nSpecial Attention: OriginPro 2022 or any latest version is needed.    ',end='')
print('(c) By XJTU-lianghui. April 5 2024\n\n')
print('Initializing...  ',end='')

import pandas as pd
import tkinter.filedialog
import tkinter.messagebox
import tkinter as tk
import os,time,sys,math,traceback
import originpro as op
import win32gui,win32con


root = tk.Tk()
root.withdraw()
# root.geometry("0x0")
root.attributes("-topmost",1)

print('初始化完成！\n')

go=False
cnt=0
while go==False:
    try:
        # top = tk.Toplevel(root)
        # top.title("选择文件")
        # top.geometry("0x0")
        print('打开文件选择窗口中。。。\n')
        file_path = tkinter.filedialog.askopenfilename(filetypes=[('CSV','.csv'),('XLSX','.xlsx'),('XLS','.xls')])
        # “TXT”, “.txt”), (“JPG”, “.jpg”), (“GIF”, “.gif”
        # top.withdraw()
        if not file_path:
            cnt+=1
            if cnt<3:
                tkinter.messagebox.showinfo(title='Yo, Check it out',message='请选择含有IV曲线数据的 Excel (CSV) 导出的原文件！')
                print('需要选择含有IV曲线数据的Excel文件！\n\n')
            else:
                tkinter.messagebox.showerror(title='达咩达咩',message='已经提示3次，是否哪里出了问题？\n\n~退出循环')
                print('已经提示3次，是否哪里出了问题？\n\n~退出循环\n')
                input('Program Aborted. Press Any Button To Close This Window...')
                os._exit(-1)
            continue
            
        filename=os.path.basename(file_path)
        basepath=os.path.dirname(file_path)
        # print(filename.split('.')[:-1])
        new_filepath=basepath+'/'+''.join(filename.split('.')[:-1])+'-for-I-V-curve.xlsx'
        new_filepath2=basepath+'/'+''.join(filename.split('.')[:-1])+'-for-I-V-curve_Origin.opju'

        IV_data_raw=pd.read_csv(file_path,encoding='gbk')
        # print(IV_data_raw)
        marker_index=IV_data_raw[IV_data_raw.iloc[:,0]=='=========='].index[0]
        IV_data=IV_data_raw.iloc[marker_index+1:,:14]
        IV_data.columns=IV_data.iloc[0]
        IV_data=IV_data.iloc[1:].reset_index(drop=True)
        print('文件读取成功！生成目标文件地址如下：')
        print(new_filepath)
        print()

        go=True
    except Exception as e:
        tkinter.messagebox.showinfo(title='Yo, Check it out',message='请选择从电脑导出的源(原)文件！而非修改后的文件！\n\n异常原因：'+str(e))
        print('请选择从电脑导出的源(原)文件！而非修改后的文件！\n\n')
        
    # top.destroy()
    time.sleep(0.1)
    
def apply_colour(value):
    value=float(value)
    if math.isnan(value):
        colour = '#00B8EA' #Blue
    elif value < 0:
        colour = '#FF0000' #Red
    else:
        colour = '#008000' #Green
    return 'color: %s' % colour

def colored_max(s):
    s=s.astype(float)
    is_max = s == s.max()
    return ['color: red;font-weight: bold' if v else '' for v in is_max]

def highlight_max(s):
    s=s.astype(float)
    is_max = s == s.max()
    return ['background: yellow;font-weight: bold' if v else '' for v in is_max]

def colored_min(s):
    s=s.astype(float)
    is_min = s == s.min()
    return ['color: green;font-weight: bold' if v else '' for v in is_min]

def apply_bold(s):
    return ['font-weight: bold']

def highlight(s):
    return ['background-color: yellow;font-weight: bold' for null in s]

raw_rows=IV_data_raw.shape[0];raw_cols=IV_data_raw.shape[1]
how_many_sets=int(raw_cols/6)
how_many_sets
IV_curve_data_raw=IV_data_raw.iloc[0:marker_index,[2]+[i*6+4 for i in range(how_many_sets)]]
new_head2=IV_curve_data_raw.iloc[0]
# new_head=[slave.strip('[]') for slave in new_head2]
# new_head=['Volt (V)']+[chr(slave+9312)+'-'+IV_data_raw.iloc[1,1+6*slave] for slave in range(how_many_sets)]
new_head=['Volt (V)']+[IV_data_raw.iloc[18,1+6*slave] for slave in range(how_many_sets)]
# new_head=['Volt (V)']+[''.join(IV_data_raw.iloc[1,1+6*slave].split('(')[-1].split(')')) for slave in range(how_many_sets)]
new_head2=['Volt (V)']+['Jsc (mA/cm^2)' for slave in range(how_many_sets)]
new_head3=[IV_data_raw.iloc[11,0]]+[IV_data_raw.iloc[11,1+6*slave] for slave in range(how_many_sets)]
IV_curve_data_raw.columns=new_head
IV_curve_data_raw=IV_curve_data_raw.iloc[1:]
IV_curve_data_raw=IV_curve_data_raw.astype(float)
# IV_curve_data=IV_curve_data_raw.iloc[:,[0,1]]

for j in range(how_many_sets):
    if new_head3[j+1]=="TRUE":
        IV_curve_data_raw.iloc[:,j+1]=IV_curve_data_raw.iloc[::-1,j+1].tolist()
    IV_curve_data_raw.iloc[:,j+1]=IV_curve_data_raw.iloc[:,j+1].map(lambda x:-x)


IV_data_cleaned=pd.DataFrame()
# IV_data_cleaned=IV_data.iloc[:][['Name','Voc (V)','Jsc (mA/cm^2)','Fill Factor (%)','Efficiency (%)']]
# IV_data_cleaned=IV_data_cleaned


IV_data_cleaned=IV_data.iloc[[0,1]][['Name','Voc (V)','Jsc (mA/cm^2)','Fill Factor (%)','Efficiency (%)']]

for idx,data in IV_data.iloc[:][['Name','Voc (V)','Jsc (mA/cm^2)','Fill Factor (%)','Efficiency (%)']].iterrows():
    target_name_split=data['Name'].split('.CH_Ref.')

    star=0
    drop_list=[]
    for idx2,data2 in IV_data_cleaned.iterrows():
        data2_split=data2['Name'].split('.CH_Ref.')

        if target_name_split[0].split('.')[0] == data2_split[0].split('.')[0] and target_name_split[1]==data2_split[1]:
            star=1
            if float(data['Efficiency (%)'])>float(data2['Efficiency (%)']):
                drop_list.append(idx2)
               
    if star==0:
        IV_data_cleaned=IV_data_cleaned.append(data)
    elif drop_list!=[]:
        IV_data_cleaned.drop(drop_list,inplace=True)
        # IV_data_cleaned.reset_index(drop=True, inplace=True)
        IV_data_cleaned=IV_data_cleaned.append(data)
    # print()


IV_data_cleaned2=IV_data_cleaned.copy()
IV_data_cleaned2.columns=['Name','Voc(V)','Jsc(mA/cm^2)','FF(%)','PCE(%)']

for idx3,data3 in IV_data_cleaned.iterrows():
    if float(data3['Fill Factor (%)'])<0:
        data3['Fill Factor (%)']=0

IV_data_cleaned.index=IV_data_cleaned.index+1
# IV_data_cleaned[['Voc (V)','Jsc (mA/cm^2)','Fill Factor (%)','Efficiency (%)']]=IV_data_cleaned[['Voc (V)','Jsc (mA/cm^2)','Fill Factor (%)','Efficiency (%)']].astype(float)
# IV_data_cleaned.dtypes



for j in range(4):
    IV_data_cleaned.iloc[:,j+1]=IV_data_cleaned.iloc[:,j+1].map(lambda x:'%.2f'%(float(x) if '---' not in str(x) else 0.0))


number_columns=['Voc (V)','Jsc (mA/cm^2)','Fill Factor (%)','Efficiency (%)']
# IV_data_cleaned.style.apply(colored_max, subset=pd.IndexSlice[:, number_columns]).apply(colored_min, subset=pd.IndexSlice[:, number_columns]).apply(highlight_max, subset=[ 'Efficiency (%)'])


CellIndexList=[]
TestLocList=[]
for everyName in IV_data_cleaned['Name']:
    slave=everyName.split('.CH_Ref.')[0].split('.')[0].split('-')
    CellIndexList.append(int(slave[0]))
    TestLocList.append(int(slave[1]))
    # TestLocList.append(int(slave[1].replace('3','4')))
IV_data_cleaned['Cell-Index']=CellIndexList
IV_data_cleaned['TestLoc']=TestLocList


def highlight_best_PCE_row(s,target):
    # print(type(s))
    # print(s.index)
    # for i in s.index:
    #     print(i,target,i == target)
    # print(['background-color: yellow;font-weight: bold' if i == target-1 else '' for i in s.index])

    return ['background-color: yellow;font-weight: bold' if i == target else '' for i in s.index]

target_row=IV_data_cleaned.iloc[:,1:].astype(float).idxmax()['Efficiency (%)']
call_highlight_best_PCE_row=lambda s:highlight_best_PCE_row(s,target_row)

class renamer():
    def __init__(self):
        self.d = dict()

    def __call__(self, x):
        if x not in self.d:
            self.d[x] = 0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])
 
# IV_curve_data_raw.rename(columns=renamer())
IV_curve_data_raw=IV_curve_data_raw.rename(columns=renamer())
# IV_curve_data_raw.columns[IV_curve_data_raw.columns.duplicated(keep=False)]
# IV_curve_data_raw.rename(IV_curve_data_raw.columns[IV_curve_data_raw.columns.duplicated(keep=False)])
# IV_curve_data_raw.rename(columns=lambda c: IV_curve_data_raw[c].pop(0) if c in IV_curve_data_raw.keys() else c)
origin_wks_data=pd.DataFrame({'Volt (V)':IV_curve_data_raw.iloc[:,0],'Jsc (mA/cm^2)':IV_curve_data_raw.iloc[:,target_row]})



# IV_data_cleaned=IV_data_cleaned.iloc[:,1:].astype(float).style.bar(subset=['Efficiency (%)'], color='#d65f5f')
# IV_data_cleaned.apply(call_highlight_best_PCE_row, axis=0).apply(colored_max, subset=pd.IndexSlice[:, number_columns]).apply(colored_min, subset=pd.IndexSlice[:, number_columns]).apply(highlight_max, subset=[ 'Efficiency (%)'])
IV_data_cleaned=IV_data_cleaned.style.apply(call_highlight_best_PCE_row, axis=0).apply(colored_max, subset=pd.IndexSlice[:, number_columns]).apply(colored_min, subset=pd.IndexSlice[:, number_columns]).apply(highlight_max, subset=[ 'Efficiency (%)']).set_properties(**{'text-align':'left'})
print("最大值最小值寻找, 高亮PCE处理\n")

IV_data_cleaned2=IV_data_cleaned2.iloc[:,1:].astype(float).style.bar(align='mean', color=['#d65f5f', '#5fba7d']).set_properties(**{'text-align':'left'})
# IV_data_cleaned2


IV_curve_data_raw=IV_curve_data_raw.style.apply(highlight,axis=0,subset=[IV_curve_data_raw.columns[target_row]]).apply(highlight,axis=0,subset=['Volt (V)'])
# IV_curve_data_raw

go=False
# go=True
cnt=0
while go==False:
    try:
        print('尝试写入文件中。。。\n')
        writer=pd.ExcelWriter(new_filepath)
        rows_shift=0

        try:
            pd.DataFrame([new_head2,new_head3]).style.apply(apply_bold, axis=0).set_properties(**{'text-align':'center'}).to_excel(writer)
            rows_shift+=3
            IV_curve_data_raw.to_excel(writer,startrow=rows_shift)
            rows_shift+=marker_index+2
            IV_data_cleaned.to_excel(writer,startrow=rows_shift)
            IV_data_cleaned2.to_excel(writer,startrow=rows_shift,startcol=9,index=False)

            wb = writer.book
            ws = wb.active
            for i, col in enumerate(ws.columns):
                max_length = 0
                column = col[0].column_letter  # 获取列字母序号
                for cell in col:
                    try:
                        # 获取单元格中最长的文本长度
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = max_length + 2  # 适配宽度多留较宽空隙
                ws.column_dimensions[column].width = adjusted_width
        except Exception as e:
            tkinter.messagebox.showinfo(title='Yo, Check it out', detail="\n\n~退出循环",message='是DataFrame输出到Excel出现的问题\n\n异常原因：'+str(e))
            writer.close()
            break

        go=True
        writer.close()
        os.startfile(new_filepath)  
        print('写入文件成功，正在打开中。。。\n')
    except Exception as e:
        top = tk.Toplevel(root)
        top.title("选择文件")
        top.withdraw()
        os.startfile(new_filepath)  
        if cnt<2:
            tkinter.messagebox.showinfo(title='Yo, Check it out', detail="\n\n已经关闭了，重试一次\nAlready Closed! I WANNA Retry~",message='旧文件被打开，无法读写。 \n请先关闭（后边这个Excel）！\n\n异常原因：'+str(e))
            print('读写IO被程序占用，请先关闭目标文件！\n')
        else:
            tkinter.messagebox.showerror(title='达咩达咩',message='已经提示3次，是否哪里出了问题？\n\n~退出循环')
            print('已经提示3次，是否哪里出了问题？\n\n~退出循环\n')
            go=True
        top.destroy()
    cnt+=1
    time.sleep(0.1)



This is a Program designed to handle IV-perovskite data enabling operating Origin Software readily.

This program will automatically import the highlight columns of the opened Excel file to draw IV-curve of PSCs with the best PCE.

Special Attention: OriginPro 2022 or any latest version is needed.    (c) By XJTU-lianghui. April 5 2024


Initializing...  初始化完成！

打开文件选择窗口中。。。

文件读取成功！生成目标文件地址如下：
C:/Users/24302/OneDrive - stu.xjtu.edu.cn/桌面/源数据/2024-4-19/all-for-I-V-curve.xlsx

最大值最小值寻找, 高亮PCE处理

尝试写入文件中。。。

写入文件成功，正在打开中。。。



In [58]:
# origin_wks_data=pd.DataFrame({'Volt (V)':IV_curve_data_raw.iloc[:,0],'Jsc (mA/cm^2)':IV_curve_data_raw.iloc[:,target_row]})
# origin_wks_data.to_csv("C:/Users/24302/OneDrive - stu.xjtu.edu.cn/桌面/Control_Origin.csv")
print('\n\nOrigin，启动！\n\n')
def origin_shutdown_exception_hook(exctype, value, traceback):
    '''Ensures Origin gets shut down if an uncaught exception'''
    op.exit()
    sys.__excepthook__(exctype, value, traceback)
if op and op.oext:
    sys.excepthook = origin_shutdown_exception_hook

op.set_show(True)


op.new(asksave=True)

book = op.find_book(type = 'w', name = 'Book1')
wks = book[0]

wks.from_df(origin_wks_data)
origin_wks_data_x_max,origin_wks_data_y_max=origin_wks_data.max()[0],origin_wks_data.max()[1]
wks.set_labels(['', IV_curve_data_raw.columns[target_row]], 'C')
graph = op.new_graph('IV-Curve')

graph[0].set_xlim(begin=0, end=origin_wks_data_x_max*1.1)
graph[0].set_ylim(begin=0, end=origin_wks_data_y_max*1.1)

plot = graph[0].add_plot(wks, coly=1, colx=0, type='y')

graph[0].set_xlim(begin=0, end=origin_wks_data_x_max*1.1,step=0.2)

graph[0].set_ylim(begin=0, end=origin_wks_data_y_max*1.1,step=5)

plot.set_cmd('-c 15','-d 0')
plot.symbol_kind=20


# new_filepath2=new_filepath2.replace('/','\\')
top = tk.Toplevel(root)
top.title("选择文件")
top.withdraw()
if(op.save(new_filepath2.replace('/','\\'))):
    # os.startfile(new_filepath)
    tkinter.messagebox.showinfo(title='Nyeh heh heh', detail=new_filepath2,message='Origin文件保存成功！文件地址：\n\n')
    print('Origin文件保存成功！文件地址：\n\n')
    print(new_filepath2)
    # if op.oext:
    #     op.exit()
else:
    tkinter.messagebox.showinfo(title='Oh my gosh !!',detail='\n\n可以手动点击保存~~',message='Origin文件保存失败！！是哪里出了问题呢？~\n\n')
    print('Origin文件保存失败！！是哪里出了问题呢？~\n\n')
top.destroy()


hwnd_title = dict() 
def get_all_hwnd(hwnd,mouse):
    if win32gui.IsWindow(hwnd) and win32gui.IsWindowEnabled(hwnd) and win32gui.IsWindowVisible(hwnd):
        hwnd_title.update({hwnd:win32gui.GetWindowText(hwnd)})
win32gui.EnumWindows(get_all_hwnd, 0)

for h,t in hwnd_title.items():
    if t != "":
        if "Origin".lower() in t.lower():
            print('\n\nWindow Handle Found !',([h], [t]))
            win32gui.ShowWindow(h, win32con.SW_SHOWNORMAL)
            win32gui.SetForegroundWindow(h)
# if op.oext:
#     op.exit()

op.detach()
# print(op.open(new_filepath2))


input('\n\n\nPress Any Button To Close This Window...')



Origin，启动！


Origin文件保存成功！文件地址：


C:/Users/24302/OneDrive - stu.xjtu.edu.cn/桌面/源数据/2024-4-19/all-for-I-V-curve_Origin.opju


Window Handle Found ! ([7416690], ['all-for-I-V-curve_Origin.opju - C:\\Users\\24302\\OneDrive - stu.xjtu.edu.cn\\桌面\\源数据\\2024-4-19\\ - /Folder1/ - OriginPro 2022'])


''

In [59]:
# input('Press Any Button To Close This Window...')
# jupyter nbconvert --to python "C:\Users\24302\OneDrive - stu.xjtu.edu.cn\桌面\Perovskite-I-V-curve.ipynb"
# pyinstaller -F "C:\Users\24302\OneDrive - stu.xjtu.edu.cn\桌面\Perovskite-I-V-curve.py"